In [111]:
import logging

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Loading packages ...")
import os
import sys
import time
import pickle
import json

# 3rd party packages

#from tqdm import tqdm
# since we are using it in jupyter notebook
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Project modules
from options import Options
from running import setup, pipeline_factory, validate, check_progress, NEG_METRICS
from utils import utils
from datasets.data import data_factory, Normalizer
from datasets.datasplit import split_dataset
from models.ts_transformer import model_factory
from models.loss import get_loss_module
from optimizers import get_optimizer


2023-05-04 16:57:14,091 | INFO : Loading packages ...


In [112]:

def main(config):

    total_epoch_time = 0
    total_eval_time = 0

    total_start_time = time.time()

    # Add file logging besides stdout
    file_handler = logging.FileHandler(os.path.join(config['output_dir'], 'output.log'))
    logger.addHandler(file_handler)

    logger.info('Running:\n{}\n'.format(' '.join(sys.argv)))  # command used to run

    if config['seed'] is not None:
        torch.manual_seed(config['seed'])

    device = torch.device('cuda' if (torch.cuda.is_available() and config['gpu'] != '-1') else 'cpu')
    logger.info("Using device: {}".format(device))
    if device == 'cuda':
        logger.info("Device index: {}".format(torch.cuda.current_device()))

    # Build data
    logger.info("Loading and preprocessing data ...")
    data_class = data_factory[config['data_class']]
    my_data = data_class(config['data_dir'], pattern=config['pattern'], n_proc=config['n_proc'], limit_size=config['limit_size'], config=config)
    feat_dim = my_data.feature_df.shape[1]  # dimensionality of data features
    if config['task'] == 'classification':
        validation_method = 'StratifiedShuffleSplit'
        labels = my_data.labels_df.values.flatten()
    else:
        validation_method = 'ShuffleSplit'
        labels = None

    # Split dataset
    test_data = my_data
    test_indices = None  # will be converted to empty list in `split_dataset`, if also test_set_ratio == 0
    val_data = my_data
    val_indices = []
    if config['test_pattern']:  # used if test data come from different files / file patterns
        test_data = data_class(config['data_dir'], pattern=config['test_pattern'], n_proc=-1, config=config)
        test_indices = test_data.all_IDs
    if config['test_from']:  # load test IDs directly from file, if available, otherwise use `test_set_ratio`. Can work together with `test_pattern`
        test_indices = list(set([line.rstrip() for line in open(config['test_from']).readlines()]))
        try:
            test_indices = [int(ind) for ind in test_indices]  # integer indices
        except ValueError:
            pass  # in case indices are non-integers
        logger.info("Loaded {} test IDs from file: '{}'".format(len(test_indices), config['test_from']))
    if config['val_pattern']:  # used if val data come from different files / file patterns
        val_data = data_class(config['data_dir'], pattern=config['val_pattern'], n_proc=-1, config=config)
        val_indices = val_data.all_IDs

    # Note: currently a validation set must exist, either with `val_pattern` or `val_ratio`
    # Using a `val_pattern` means that `val_ratio` == 0 and `test_ratio` == 0
    if config['val_ratio'] > 0:
        train_indices, val_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                                 validation_method=validation_method,
                                                                 n_splits=1,
                                                                 validation_ratio=config['val_ratio'],
                                                                 test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                                 test_indices=test_indices,
                                                                 random_seed=1337,
                                                                 labels=labels)
        train_indices = train_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
        val_indices = val_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
    else:
        train_indices = my_data.all_IDs
        if test_indices is None:
            test_indices = []

    logger.info("{} samples may be used for training".format(len(train_indices)))
    logger.info("{} samples will be used for validation".format(len(val_indices)))
    logger.info("{} samples will be used for testing".format(len(test_indices)))

    with open(os.path.join(config['output_dir'], 'data_indices.json'), 'w') as f:
        try:
            json.dump({'train_indices': list(map(int, train_indices)),
                       'val_indices': list(map(int, val_indices)),
                       'test_indices': list(map(int, test_indices))}, f, indent=4)
        except ValueError:  # in case indices are non-integers
            json.dump({'train_indices': list(train_indices),
                       'val_indices': list(val_indices),
                       'test_indices': list(test_indices)}, f, indent=4)

    # Pre-process features
    normalizer = None
    if config['norm_from']:
        with open(config['norm_from'], 'rb') as f:
            norm_dict = pickle.load(f)
        normalizer = Normalizer(**norm_dict)
    elif config['normalization'] is not None:
        normalizer = Normalizer(config['normalization'])
        my_data.feature_df.loc[train_indices] = normalizer.normalize(my_data.feature_df.loc[train_indices])
        if not config['normalization'].startswith('per_sample'):
            # get normalizing values from training set and store for future use
            norm_dict = normalizer.__dict__
            with open(os.path.join(config['output_dir'], 'normalization.pickle'), 'wb') as f:
                pickle.dump(norm_dict, f, pickle.HIGHEST_PROTOCOL)
    if normalizer is not None:
        if len(val_indices):
            val_data.feature_df.loc[val_indices] = normalizer.normalize(val_data.feature_df.loc[val_indices])
        if len(test_indices):
            test_data.feature_df.loc[test_indices] = normalizer.normalize(test_data.feature_df.loc[test_indices])

    # Create model
    logger.info("Creating model ...")
    model = model_factory(config, my_data)

    if config['freeze']:
        for name, param in model.named_parameters():
            if name.startswith('output_layer'):
                param.requires_grad = True
            else:
                param.requires_grad = False

    logger.info("Model:\n{}".format(model))
    logger.info("Total number of parameters: {}".format(utils.count_parameters(model)))
    logger.info("Trainable parameters: {}".format(utils.count_parameters(model, trainable=True)))


    # Initialize optimizer

    if config['global_reg']:
        weight_decay = config['l2_reg']
        output_reg = None
    else:
        weight_decay = 0
        output_reg = config['l2_reg']

    optim_class = get_optimizer(config['optimizer'])
    optimizer = optim_class(model.parameters(), lr=config['lr'], weight_decay=weight_decay)

    start_epoch = 0
    lr_step = 0  # current step index of `lr_step`
    lr = config['lr']  # current learning step
    # Load model and optimizer state
    if args.load_model:
        model, optimizer, start_epoch = utils.load_model(model, config['load_model'], optimizer, config['resume'],
                                                         config['change_output'],
                                                         config['lr'],
                                                         config['lr_step'],
                                                         config['lr_factor'])
    model.to(device)

    loss_module = get_loss_module(config)

    if config['test_only'] == 'testset':  # Only evaluate and skip training
        dataset_class, collate_fn, runner_class = pipeline_factory(config)
        test_dataset = dataset_class(test_data, test_indices)

        test_loader = DataLoader(dataset=test_dataset,
                                 batch_size=config['batch_size'],
                                 shuffle=False,
                                 num_workers=config['num_workers'],
                                 pin_memory=True,
                                 collate_fn=lambda x: collate_fn(x, max_len=model.max_len))
        test_evaluator = runner_class(model, test_loader, device, loss_module,
                                            print_interval=config['print_interval'], console=config['console'])
        aggr_metrics_test, per_batch_test = test_evaluator.evaluate(keep_all=True)
        print_str = 'Test Summary: '
        for k, v in aggr_metrics_test.items():
            print_str += '{}: {:8f} | '.format(k, v)
        logger.info(print_str)
        return
    
    # Initialize data generators
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    val_dataset = dataset_class(val_data, val_indices)

    val_loader = DataLoader(dataset=val_dataset,
                            batch_size=config['batch_size'],
                            shuffle=False,
                            num_workers=config['num_workers'],
                            pin_memory=True,
                            collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    train_dataset = dataset_class(my_data, train_indices)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=config['batch_size'],
                              shuffle=True,
                              num_workers=config['num_workers'],
                              pin_memory=True,
                              collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    trainer = runner_class(model, train_loader, device, loss_module, optimizer, l2_reg=output_reg,
                                 print_interval=config['print_interval'], console=config['console'])
    val_evaluator = runner_class(model, val_loader, device, loss_module,
                                       print_interval=config['print_interval'], console=config['console'])

    tensorboard_writer = SummaryWriter(config['tensorboard_dir'])

    best_value = 1e16 if config['key_metric'] in NEG_METRICS else -1e16  # initialize with +inf or -inf depending on key metric
    metrics = []  # (for validation) list of lists: for each epoch, stores metrics like loss, ...
    best_metrics = {}

    # Evaluate on validation before training
    aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config, best_metrics,
                                                          best_value, epoch=0)
    metrics_names, metrics_values = zip(*aggr_metrics_val.items())
    metrics.append(list(metrics_values))

    logger.info('Starting training...')
    for epoch in tqdm(range(start_epoch + 1, config["epochs"] + 1), desc='Training Epoch', leave=False):
        mark = epoch if config['save_all'] else 'last'
        epoch_start_time = time.time()
        aggr_metrics_train = trainer.train_epoch(epoch)  # dictionary of aggregate epoch metrics
        epoch_runtime = time.time() - epoch_start_time
        print()
        print_str = 'Epoch {} Training Summary: '.format(epoch)
        for k, v in aggr_metrics_train.items():
            tensorboard_writer.add_scalar('{}/train'.format(k), v, epoch)
            print_str += '{}: {:8f} | '.format(k, v)
        logger.info(print_str)
        logger.info("Epoch runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(epoch_runtime)))
        total_epoch_time += epoch_runtime
        avg_epoch_time = total_epoch_time / (epoch - start_epoch)
        avg_batch_time = avg_epoch_time / len(train_loader)
        avg_sample_time = avg_epoch_time / len(train_dataset)
        logger.info("Avg epoch train. time: {} hours, {} minutes, {} seconds".format(*utils.readable_time(avg_epoch_time)))
        logger.info("Avg batch train. time: {} seconds".format(avg_batch_time))
        logger.info("Avg sample train. time: {} seconds".format(avg_sample_time))

        # evaluate if first or last epoch or at specified interval
        if (epoch == config["epochs"]) or (epoch == start_epoch + 1) or (epoch % config['val_interval'] == 0):
            aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config,
                                                                  best_metrics, best_value, epoch)
            metrics_names, metrics_values = zip(*aggr_metrics_val.items())
            metrics.append(list(metrics_values))

        utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(mark)), epoch, model, optimizer)

        # Learning rate scheduling
        if epoch == config['lr_step'][lr_step]:
            utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(epoch)), epoch, model, optimizer)
            lr = lr * config['lr_factor'][lr_step]
            if lr_step < len(config['lr_step']) - 1:  # so that this index does not get out of bounds
                lr_step += 1
            logger.info('Learning rate updated to: ', lr)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        # Difficulty scheduling
        if config['harden'] and check_progress(epoch):
            train_loader.dataset.update()
            val_loader.dataset.update()

    # Export evolution of metrics over epochs
    header = metrics_names
    metrics_filepath = os.path.join(config["output_dir"], "metrics_" + config["experiment_name"] + ".xls")
    book = utils.export_performance_metrics(metrics_filepath, metrics, header, sheet_name="metrics")

    # Export record metrics to a file accumulating records from all experiments
    utils.register_record(config["records_file"], config["initial_timestamp"], config["experiment_name"],
                          best_metrics, aggr_metrics_val, comment=config['comment'])

    logger.info('Best {} was {}. Other metrics: {}'.format(config['key_metric'], best_value, best_metrics))
    logger.info('All Done!')

    total_runtime = time.time() - total_start_time
    logger.info("Total runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(total_runtime)))

    return best_value


In [114]:
text = "--output_dir ../experiments/ --comment 'regression_from_Scratch' \
        --name pm25_fromScratch_Regression --records_file Regression_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_pattern TEST --epochs 100 --lr 0.001 --optimizer RAdam \
        --pos_encoding learnable --task regression"

text = "--output_dir ../experiments/ --comment 'regression_from_Scratch' \
        --name pm25_fromScratch_Regression_test --records_file Regression_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_pattern TEST --epochs 100 --lr 0.001 --optimizer RAdam \
        --pos_encoding learnable --task regression"

### Pretrain
python src/main.py --output_dir experiments --comment "pretraining through imputation" --name $1_pretrained 
--records_file Imputation_records.xls --data_dir /path/to/$1/ --data_class tsra --pattern TRAIN --val_ratio 0.2 --epochs 700 --lr 0.001 --optimizer RAdam --batch_size 32 --pos_encoding learnable --d_model 128

### Finetune
python src/main.py --output_dir experiments --comment "finetune for regression" --name BeijingPM25Quality_finetuned --records_file Regression_records.xls --data_dir /path/to/Datasets/Regression/BeijingPM25Quality/ --data_class tsra --pattern TRAIN --val_pattern TEST  --epochs 200 --lr 0.001 --optimizer RAdam --pos_encoding learnable --d_model 128 --load_model path/to/BeijingPM25Quality_pretrained/checkpoints/model_best.pth --task regression --change_output --batch_size 128


In [115]:
# Pretrain
'''
text = "--output_dir ../experiments/ --comment 'pretraining_through_imputation' \
        --name pm25_pretrained --records_file Imputation_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_ratio 0.2 --epochs 20 --lr 0.001 --optimizer RAdam \
        --batch_size 32 --pos_encoding learnable --d_model 128"
'''


# Finetune
text = "--output_dir ../experiments --comment 'finetune_for_regression' \
        --name BeijingPM25Quality_finetuned --records_file Regression_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_pattern TEST  --epochs 100 --lr 0.001 --optimizer RAdam \
        --pos_encoding learnable --d_model 128 \
        --load_model ../experiments/pm25_pretrained_2023-05-04_11-01-18_AhP/checkpoints/model_best.pth \
        --task regression --change_output --batch_size 128"

In [116]:
input_text = text.split()
#input_text

In [117]:
args = Options().parse(input_text)  # `argsparse` object
#config = setup(args)  # configuration dictionary

In [118]:
import parser

In [119]:
#all_defaults = {}
#for key in vars(args):
#    print(key)

In [120]:
config = setup(args)

2023-05-04 16:57:42,472 | INFO : Stored configuration file in '../experiments/BeijingPM25Quality_finetuned_2023-05-04_16-57-42_wdT'


In [121]:
main(config)

2023-05-04 16:57:43,523 | INFO : Running:
/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/tianyi/.local/share/jupyter/runtime/kernel-796aefab-8958-4f7c-bb18-fec8d264f221.json

2023-05-04 16:57:43,524 | INFO : Using device: cuda
2023-05-04 16:57:43,525 | INFO : Loading and preprocessing data ...
11942it [00:24, 488.60it/s]
5072it [00:10, 487.22it/s]
2023-05-04 16:58:50,473 | INFO : 11918 samples may be used for training
2023-05-04 16:58:50,474 | INFO : 5048 samples will be used for validation
2023-05-04 16:58:50,474 | INFO : 0 samples will be used for testing
2023-05-04 16:58:50,689 | INFO : Creating model ...
2023-05-04 16:58:50,694 | INFO : Model:
TSTransformerEncoderClassiregressor(
  (project_inp): Linear(in_features=9, out_features=128, bias=True)
  (pos_enc): LearnablePositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerBat

Loaded model from ../experiments/pm25_pretrained_2023-05-04_11-01-18_AhP/checkpoints/model_best.pth. Epoch: 20
Evaluating Epoch 0   0.0% | batch:         0 of        40	|	loss: 34523.5
Evaluating Epoch 0   2.5% | batch:         1 of        40	|	loss: 10962
Evaluating Epoch 0   5.0% | batch:         2 of        40	|	loss: 49514
Evaluating Epoch 0   7.5% | batch:         3 of        40	|	loss: 30578.3
Evaluating Epoch 0  10.0% | batch:         4 of        40	|	loss: 11949.3
Evaluating Epoch 0  12.5% | batch:         5 of        40	|	loss: 17943.6
Evaluating Epoch 0  15.0% | batch:         6 of        40	|	loss: 44269.7
Evaluating Epoch 0  17.5% | batch:         7 of        40	|	loss: 18911.3
Evaluating Epoch 0  20.0% | batch:         8 of        40	|	loss: 13043.2
Evaluating Epoch 0  22.5% | batch:         9 of        40	|	loss: 33830.7
Evaluating Epoch 0  25.0% | batch:        10 of        40	|	loss: 28445.7
Evaluating Epoch 0  27.5% | batch:        11 of        40	|	loss: 15385.7
Evalu

2023-05-04 16:58:51,290 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 0.4556422233581543 seconds

2023-05-04 16:58:51,290 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 0.5502525597442816 seconds
2023-05-04 16:58:51,291 | INFO : Avg batch val. time: 0.01375631399360704 seconds
2023-05-04 16:58:51,292 | INFO : Avg sample val. time: 0.0001090040728495011 seconds
2023-05-04 16:58:51,292 | INFO : Epoch 0 Validation Summary: epoch: 0.000000 | loss: 31395.954487 | 
/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/numpy/lib/npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
2023-05-04 16:58:51,304 | INFO : Starting training...


Evaluating Epoch 0  57.5% | batch:        23 of        40	|	loss: 30231.1
Evaluating Epoch 0  60.0% | batch:        24 of        40	|	loss: 12199
Evaluating Epoch 0  62.5% | batch:        25 of        40	|	loss: 54077.7
Evaluating Epoch 0  65.0% | batch:        26 of        40	|	loss: 50364.5
Evaluating Epoch 0  67.5% | batch:        27 of        40	|	loss: 17486.7
Evaluating Epoch 0  70.0% | batch:        28 of        40	|	loss: 38107.6
Evaluating Epoch 0  72.5% | batch:        29 of        40	|	loss: 54040
Evaluating Epoch 0  75.0% | batch:        30 of        40	|	loss: 17143.6
Evaluating Epoch 0  77.5% | batch:        31 of        40	|	loss: 17641.4
Evaluating Epoch 0  80.0% | batch:        32 of        40	|	loss: 54738
Evaluating Epoch 0  82.5% | batch:        33 of        40	|	loss: 26178
Evaluating Epoch 0  85.0% | batch:        34 of        40	|	loss: 12782.8
Evaluating Epoch 0  87.5% | batch:        35 of        40	|	loss: 51777.4
Evaluating Epoch 0  90.0% | batch:        36 o

Training Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

...

2023-05-04 17:02:14,590 | INFO : Epoch 100 Training Summary: epoch: 100.000000 | loss: 915.600737 | 
2023-05-04 17:02:14,591 | INFO : Epoch runtime: 0.0 hours, 0.0 minutes, 1.7670378684997559 seconds

2023-05-04 17:02:14,592 | INFO : Avg epoch train. time: 0.0 hours, 0.0 minutes, 1.7819194674491883 seconds
2023-05-04 17:02:14,592 | INFO : Avg batch train. time: 0.018956590079246684 seconds
2023-05-04 17:02:14,593 | INFO : Avg sample train. time: 0.0001495149746139611 seconds
2023-05-04 17:02:14,593 | INFO : Evaluating on validation set ...


Training Epoch 100  94.7% | batch:        89 of        94	|	loss: 822.854
Training Epoch 100  95.7% | batch:        90 of        94	|	loss: 673.4
Training Epoch 100  96.8% | batch:        91 of        94	|	loss: 779.798
Training Epoch 100  97.9% | batch:        92 of        94	|	loss: 662.252
Training Epoch 100  98.9% | batch:        93 of        94	|	loss: 1357.74

Evaluating Epoch 100   0.0% | batch:         0 of        40	|	loss: 7752.27
Evaluating Epoch 100   2.5% | batch:         1 of        40	|	loss: 1201.23
Evaluating Epoch 100   5.0% | batch:         2 of        40	|	loss: 3293.98
Evaluating Epoch 100   7.5% | batch:         3 of        40	|	loss: 6965.75
Evaluating Epoch 100  10.0% | batch:         4 of        40	|	loss: 2329.23
Evaluating Epoch 100  12.5% | batch:         5 of        40	|	loss: 2301.91
Evaluating Epoch 100  15.0% | batch:         6 of        40	|	loss: 8325.72
Evaluating Epoch 100  17.5% | batch:         7 of        40	|	loss: 3693.76
Evaluating Epoch 100  2

2023-05-04 17:02:15,033 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 0.4390065670013428 seconds

2023-05-04 17:02:15,034 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 0.5260670263442635 seconds
2023-05-04 17:02:15,034 | INFO : Avg batch val. time: 0.013151675658606587 seconds
2023-05-04 17:02:15,035 | INFO : Avg sample val. time: 0.00010421296084474316 seconds
2023-05-04 17:02:15,036 | INFO : Epoch 100 Validation Summary: epoch: 100.000000 | loss: 4193.033844 | 
2023-05-04 17:02:15,060 | INFO : Exported per epoch performance metrics in '../experiments/BeijingPM25Quality_finetuned_2023-05-04_16-57-42_wdT/metrics_BeijingPM25Quality_finetuned.xls'
2023-05-04 17:02:15,065 | INFO : Exported performance record to 'Regression_records.xls'
2023-05-04 17:02:15,065 | INFO : Best loss was 3424.168098751981. Other metrics: OrderedDict([('epoch', 14), ('loss', 3424.168098751981)])
2023-05-04 17:02:15,066 | INFO : All Done!
2023-05-04 17:02:15,067 | INFO : Total runtime: 0.0 hours, 4.0 min

Evaluating Epoch 100  75.0% | batch:        30 of        40	|	loss: 1842.61
Evaluating Epoch 100  77.5% | batch:        31 of        40	|	loss: 1499.66
Evaluating Epoch 100  80.0% | batch:        32 of        40	|	loss: 8207.79
Evaluating Epoch 100  82.5% | batch:        33 of        40	|	loss: 6172.15
Evaluating Epoch 100  85.0% | batch:        34 of        40	|	loss: 1064.73
Evaluating Epoch 100  87.5% | batch:        35 of        40	|	loss: 5565.65
Evaluating Epoch 100  90.0% | batch:        36 of        40	|	loss: 6693.4
Evaluating Epoch 100  92.5% | batch:        37 of        40	|	loss: 2868.13
Evaluating Epoch 100  95.0% | batch:        38 of        40	|	loss: 3696.24
Evaluating Epoch 100  97.5% | batch:        39 of        40	|	loss: 11637.8



3424.168098751981